# Lab 1 - Differential Expression analysis
## Differential expression analysis, an intuition

In [156]:
import pandas as pd
import numpy as np
import seaborn as sns
from ipywidgets import interact, interact_manual
from ipywidgets import IntSlider, FloatSlider

plt.rcParams["figure.figsize"] = (20,10)

def generate_data(distance,dispersion, n_points):
    dispersion = [[dispersion,0],[0,dispersion]]
    cond1 = np.random.multivariate_normal([0,0], dispersion, n_points)
    cond2 = np.random.multivariate_normal([distance,0], dispersion, n_points)
    data1 = pd.DataFrame(cond1, columns=['x','y'])
    data1['Condition'] = 1
    data2 = pd.DataFrame(cond2, columns=['x','y'])
    data2['Condition'] = 2
    data = pd.concat([data1,data2])
    return data

def shuffle_condition(data):
    data_s = data.copy()
    data_s['Condition'] = data['Condition'].sample(frac=1).reset_index(drop=True)
    return data_s

def plot_random(data,data_s):
    f, ax = plt.subplots(1, 2)
    r1 = np.random.permutation(2)
    colors = np.random.permutation(['red','blue']).tolist()
    sns.scatterplot(x='x', y='y', data=data, hue = 'Condition', palette=colors, ax = ax[r1[0]], legend  = False)
    sns.scatterplot(x='x', y='y', data=data_s, hue = 'Condition', palette=colors, ax = ax[r1[1]], legend = False)

def dist_plot(Distance,Dispersion,Samples):
    data = generate_data(Distance,Dispersion, Samples)
    data_s = shuffle_condition(data)
    plot_random(data,data_s)
    plt.show()

In [160]:
interact(dist_plot, Distance = FloatSlider(min=0,max=10, continuous_update=False), Dispersion = FloatSlider(min=0,max=100, continuous_update=False), Samples=IntSlider(min=10,max=500, continuous_update=False))

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='Distance', max=10.0), Float…

<function __main__.dist_plot(Distance, Dispersion, Samples)>

## How can we determine differential expression?
### The t-test

Question: 
* What assumptions about the data doest the t-test make?
* Are they compatible with gene expression data? If not, how to go around it? 